In [ ]:
# LR, DT, NB, XGBOOST Modeling - Bank Marketing Dataset

In [20]:
# importing required Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,accuracy_score,recall_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

In [21]:
#Reading Preprocessed data. First column is deleted because its index and redundant
df=pd.read_csv("PreprocessedBank.csv").drop(['Unnamed: 0'],axis=1)

In [22]:
#display first five rows
df.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,jun,mar,may,nov,oct,sep,failure,success,unknown,deposit
0,59,1,2343,0,1,5,1042,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
1,56,1,45,1,1,5,1467,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
2,41,1,1270,0,1,5,1389,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
3,55,1,2476,0,1,5,579,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
4,54,1,184,1,1,5,673,2,-1,0,...,0,0,1,0,0,0,0,0,1,1


In [23]:
#Standardisation of Data
scaler=StandardScaler()
scaler.fit(df.drop('deposit',axis=1))
scaled_features=scaler.transform(df.drop('deposit',axis=1))
df_feat=pd.DataFrame(scaled_features,columns=df.columns[:-1])
df_feat.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,jul,jun,mar,may,nov,oct,sep,failure,success,unknown
0,1.491505,0.123617,0.252525,-1.055280,0.387923,-1.265746,1.930226,-0.554168,-0.481184,-0.36326,...,-0.396136,-0.350625,-0.159228,1.718298,-0.303775,-0.190781,-0.171522,-0.35159,-0.325782,0.583626
1,1.239676,0.123617,-0.459974,0.947616,0.387923,-1.265746,3.154612,-0.554168,-0.481184,-0.36326,...,-0.396136,-0.350625,-0.159228,1.718298,-0.303775,-0.190781,-0.171522,-0.35159,-0.325782,0.583626
2,-0.019470,0.123617,-0.080160,-1.055280,0.387923,-1.265746,2.929901,-0.554168,-0.481184,-0.36326,...,-0.396136,-0.350625,-0.159228,1.718298,-0.303775,-0.190781,-0.171522,-0.35159,-0.325782,0.583626
3,1.155733,0.123617,0.293762,-1.055280,0.387923,-1.265746,0.596366,-0.554168,-0.481184,-0.36326,...,-0.396136,-0.350625,-0.159228,1.718298,-0.303775,-0.190781,-0.171522,-0.35159,-0.325782,0.583626
4,1.071790,0.123617,-0.416876,0.947616,0.387923,-1.265746,0.867171,-0.186785,-0.481184,-0.36326,...,-0.396136,-0.350625,-0.159228,1.718298,-0.303775,-0.190781,-0.171522,-0.35159,-0.325782,0.583626


Notes :
1. Algorithms like KNN, Naive Bayes, Logistic Regression and SVM require Standardised Data
2. Tree-based Algorithms are scale independent
3. Standardisation basically subtracts the mean and divides the std deviation of the column from each value

# Train Test Split

In [26]:
# 30% Data is set aside for tesing
X_train,X_test,y_train,y_test=train_test_split(scaled_features,df['deposit'],test_size=0.30, random_state=3)

# Applying Naive Bayes

In [28]:
# Training Naive bayes
# Finding Accuracy, AUC, False Positive Rate, True positive rate, confusion matrix and classificatio report

gnb = GaussianNB()
gnb.fit(X_train, y_train)
pred = gnb.predict(X_test)
accNB = accuracy_score(y_test, pred)
y_pred_prob = gnb.predict_proba(X_test)
aucScoreNB = roc_auc_score(y_test,  y_pred_prob[:,1])
fprNB, tprNB, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for NB is ",aucScoreNB)
print("Test Accuracy score for NB is ",accuracy_score(y_test, pred))
predT=gnb.predict(X_train)
print("Train Accuracy score for NB is ",accuracy_score(y_train, predT))
#print("Best parameters for NB are ",gnb.best_params_)
print("Recall score for NB is ",recall_score(y_test, pred))

[[1517  257]
 [ 737  838]]
              precision    recall  f1-score   support

           0       0.67      0.86      0.75      1774
           1       0.77      0.53      0.63      1575

    accuracy                           0.70      3349
   macro avg       0.72      0.69      0.69      3349
weighted avg       0.72      0.70      0.69      3349

AUC score for NB is  0.7965192104650956
Test Accuracy score for NB is  0.7031949835771872
Train Accuracy score for NB is  0.7317291693331627
Recall score for NB is  0.532063492063492


Notes:

1. Naive Bayes gives an accuracy of 0.70, which is not very good
2. There is very little difference between train and test accuracy, so its not overfitting or underfitting
3. Recall is 0.53, which is very poor

# Applying Logistic Regression

In [31]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 10000]
    }
]
logModel = LogisticRegression()

Notes:

1. Hyperparameters for Logistic Regression
2. Penalty indicates which regularization to use
3. C decides how much weightage is given to Regularization
4. Solver is the technique used internally
5. max_iter is the number of iterations for convergence

In [33]:
#GridsearchCV tries all possible combinations of hyperparameters to find best accuracy

clf = GridSearchCV(logModel, param_grid = param_grid,scoring='accuracy', cv = 5 )
best_clf = clf.fit(X_train,y_train)

In [34]:
# Training Logistic Regression
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report

pred = best_clf.predict(X_test)
accLR = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_test)
aucScoreLR = roc_auc_score(y_test,  y_pred_prob[:,1])
fprLR, tprLR, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for LR is ",aucScoreLR)
print("Test Accuracy score for LR is ",accuracy_score(y_test, pred))
predT=best_clf.predict(X_train)
print("Train Accuracy score for LR is ",accuracy_score(y_train, predT))
print("Best parameters for accuracy of LR are ",best_clf.best_params_)

[[1489  285]
 [ 322 1253]]
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1774
           1       0.81      0.80      0.81      1575

    accuracy                           0.82      3349
   macro avg       0.82      0.82      0.82      3349
weighted avg       0.82      0.82      0.82      3349

AUC score for LR is  0.8913383797712997
Test Accuracy score for LR is  0.818751866228725
Train Accuracy score for LR is  0.8306668373224113
Best parameters for accuracy of LR are  {'C': 0.615848211066026, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}


In [35]:
# Training Logistic Regression for recall

clfR = GridSearchCV(logModel, param_grid = param_grid,scoring='recall', cv = 5 )
best_clfR = clfR.fit(X_train,y_train)

In [36]:
# print recall and best parameters

predR = best_clfR.predict(X_test)
predRT=best_clfR.predict(X_train)
recallLR=recall_score(y_test, predR)
print("Test Recall score for LR is ",recallLR)
print("Train recall score for LR is ",recall_score(y_train, predRT))
print("Best parameters for recall of LR are ",best_clfR.best_params_)

Test Recall score for LR is  0.7949206349206349
Train recall score for LR is  0.7977921378567582
Best parameters for recall of LR are  {'C': 29.763514416313132, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}


Notes:

1. Logistic Regression gives accuracy of 0.81 which is fairly good
2. There is very less difference between train and test accuracy so its not overfitting or underfitting
3. Recall is 0.79 which is not very good

# Logistic Regression with Polynomial Features (degree=2)

In [39]:
# Logistic regression with polynomial features works better with normalozation instead of standardization
# so read file again

df=pd.read_csv("PreprocessedBank.csv").drop(['Unnamed: 0'],axis=1)
dfX=df.drop('deposit',axis=1)
df.head()


,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,jun,mar,may,nov,oct,sep,failure,success,unknown,deposit
0,59,1,2343,0,1,5,1042,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
1,56,1,45,1,1,5,1467,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
2,41,1,1270,0,1,5,1389,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
3,55,1,2476,0,1,5,579,1,-1,0,...,0,0,1,0,0,0,0,0,1,1
4,54,1,184,1,1,5,673,2,-1,0,...,0,0,1,0,0,0,0,0,1,1


In [40]:
# Minmaxscaler is used to normalise data

scaler = MinMaxScaler()
bankMM = scaler.fit_transform(dfX)
bankMM = pd.DataFrame(bankMM, columns=dfX.columns)
bankMM.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,jul,jun,mar,may,nov,oct,sep,failure,success,unknown
0,0.532468,1.0,0.104371,0.0,1.0,0.133333,0.268110,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.493506,1.0,0.078273,1.0,1.0,0.133333,0.377675,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.298701,1.0,0.092185,0.0,1.0,0.133333,0.357566,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.480519,1.0,0.105882,0.0,1.0,0.133333,0.148750,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.467532,1.0,0.079851,1.0,1.0,0.133333,0.172983,0.016129,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [41]:
# train test split (70:30)
X_train,X_test,y_train,y_test=train_test_split(bankMM,df['deposit'],test_size=0.30, random_state=3)

In [42]:
# creating polynomial features with degree 2
poly2 = PolynomialFeatures(degree=2)
X_trainP=poly2.fit_transform(X_train)

In [43]:
# Hyperparameters 
param_grid = {'penalty' : ['l1', 'l2'], 'C' : [0.001,0.01,0.1,1,5,25]    }

In [44]:
#GridserchCV tries all possible combinations of hyperparameters to find best accuracy

logModel = LogisticRegression()
clf = GridSearchCV(logModel, param_grid = param_grid,scoring='accuracy',verbose=True, cv = 5,n_jobs=-1 )
best_clf = clf.fit(X_trainP,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [45]:
# create polynomial features with degree 2 for test data
X_testP=poly2.fit_transform(X_test)

In [46]:
# Training Logistic Regression with polynomial features with degree 2
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
# get best parameters for retraining

pred = best_clf.predict(X_testP)
accLRP2 = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_testP)
aucScoreLRP2 = roc_auc_score(y_test,  y_pred_prob[:,1])
fprLRP2, tprLRP2, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for LR Poly2 is ",aucScoreLRP2)
print("Test Accuracy score for LR Poly2 is ",accLRP2)
predT=best_clf.predict(X_trainP)
print("Train Accuracy score for LR Poly2 is ",accuracy_score(y_train, predT))
print("Best parameters for LR are ",best_clf.best_params_)

[[1519  255]
 [ 266 1309]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      1774
           1       0.84      0.83      0.83      1575

    accuracy                           0.84      3349
   macro avg       0.84      0.84      0.84      3349
weighted avg       0.84      0.84      0.84      3349

AUC score for LR Poly2 is  0.9071301515720908
Test Accuracy score for LR Poly2 is  0.8444311734846223
Train Accuracy score for LR Poly2 is  0.8681684372200179
Best parameters for LR are  {'C': 1, 'penalty': 'l2'}


In [47]:
logModel = LogisticRegression()
clfR = GridSearchCV(logModel, param_grid = param_grid,scoring='recall',verbose=True, cv = 5,n_jobs=-1 )
best_clfR = clfR.fit(X_trainP,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [48]:
# Training Logistic Regression for recall

predR = best_clfR.predict(X_testP)
predRT=best_clfR.predict(X_trainP)
recallLRP2=recall_score(y_test, predR)
print("Test Recall score for LR with polynomial features degree 2  is ",recallLRP2)
print("Train recall score for LR with polynomial features degree 2 is ",recall_score(y_train, predRT))
print("Best parameters for recall of LR with polynomial features degree 2 are ",best_clfR.best_params_)

Test Recall score for LR with polynomial features degree 2  is  0.8304761904761905
Train recall score for LR with polynomial features degree 2 is  0.8651050080775444
Best parameters for recall of LR with polynomial features degree 2 are  {'C': 5, 'penalty': 'l2'}


Notes:
Logistic regression with polynomial features degree 2 gives accuracy of 0.84 which is quite good
There is very less difference between train and test accuracy so its not overfitting or underfitting
Recall is 0.83 which is quite good


# Applying Decision Tree

In [51]:
dfX=df.drop('deposit',axis=1)
dfX.head()
X_train,X_test,y_train,y_test=train_test_split(dfX,df['deposit'],test_size=0.30, random_state=3)

In [52]:
dt=DecisionTreeClassifier()
parameters={'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
            'min_samples_leaf':[1,2,3,4,5],
            'min_samples_split':[2,3,4,5],
            'criterion':['gini','entropy']}


In [53]:
clf = GridSearchCV(dt,parameters,scoring='accuracy',verbose=True)
best_clf = clf.fit(X_train,y_train)


Fitting 5 folds for each of 600 candidates, totalling 3000 fits


In [54]:
# Training Decision Tree
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
# get best parameters for retraining

pred = best_clf.predict(X_test)
accDT = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_test)
aucScoreDT = roc_auc_score(y_test,  y_pred_prob[:,1])
fprDT, tprDT, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for Decision Tree is ",aucScoreDT)
print("Test Accuracy score for DT is ",accuracy_score(y_test, pred))
predT=best_clf.predict(X_train)
print("Train Accuracy score for DT is ",accuracy_score(y_train, predT))
print("Best parameters for DT are ",best_clf.best_params_)

[[1385  389]
 [ 235 1340]]
              precision    recall  f1-score   support

           0       0.85      0.78      0.82      1774
           1       0.78      0.85      0.81      1575

    accuracy                           0.81      3349
   macro avg       0.81      0.82      0.81      3349
weighted avg       0.82      0.81      0.81      3349

AUC score for Decision Tree is  0.875908627261502
Test Accuracy score for DT is  0.8136757240967453
Train Accuracy score for DT is  0.8493536413669525
Best parameters for DT are  {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 3}


In [55]:
#Gridsearchcv Training Decision Tree for recall

clfR = GridSearchCV(dt,parameters,scoring='recall',verbose=True)
best_clfR = clfR.fit(X_train,y_train)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


In [56]:
# Training Decision Tree
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
# get best parameters for retraining

pred = best_clf.predict(X_test)
accDT = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_test)
aucScoreDT = roc_auc_score(y_test,  y_pred_prob[:,1])
fprDT, tprDT, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for Decision Tree is ",aucScoreDT)
print("Test Accuracy score for DT is ",accuracy_score(y_test, pred))
predT=best_clf.predict(X_train)
print("Train Accuracy score for DT is ",accuracy_score(y_train, predT))
print("Best parameters for DT are ",best_clf.best_params_)

[[1385  389]
 [ 235 1340]]
              precision    recall  f1-score   support

           0       0.85      0.78      0.82      1774
           1       0.78      0.85      0.81      1575

    accuracy                           0.81      3349
   macro avg       0.81      0.82      0.81      3349
weighted avg       0.82      0.81      0.81      3349

AUC score for Decision Tree is  0.875908627261502
Test Accuracy score for DT is  0.8136757240967453
Train Accuracy score for DT is  0.8493536413669525
Best parameters for DT are  {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 3}


In [57]:
#Gridsearchcv Training Decision Tree for recall

clfR = GridSearchCV(dt,parameters,scoring='recall',verbose=True)
best_clfR = clfR.fit(X_train,y_train)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


In [58]:
predR = best_clfR.predict(X_test)
predRT=best_clfR.predict(X_train)
recallDT=recall_score(y_test, predR)
print("Test Recall score for DT is ",recallDT)
print("Train recall score for DT is ",recall_score(y_train, predRT))
print("Best parameters for recall of DT are ",best_clfR.best_params_)

Test Recall score for DT is  0.8253968253968254
Train recall score for DT is  0.8309100700053851
Best parameters for recall of DT are  {'criterion': 'entropy', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2}


Notes:

Decision Tree gives accuracy of 0.81 which is quite good
There is very less difference between train and test accuracy so its not overfitting or underfitting
Recall is 0.82 which is quite good

# Applying XGBoost

In [ ]:
params = {
            'eta': np.arange(0.1, 0.26, 0.05),
            'min_child_weight': np.arange(1, 5, 0.5).tolist(),
            'gamma': [5],
            'subsample': np.arange(0.5, 1.0, 0.11).tolist(),
            'colsample_bytree': np.arange(0.5, 1.0, 0.11).tolist()
        }

In [ ]:
xgb_model = XGBClassifier(objective = "binary:logistic")
skf = StratifiedKFold(n_splits=10, shuffle = True)
clf = GridSearchCV(xgb_model, param_grid = params, scoring = 'accuracy',cv = skf.split(X_train, y_train),verbose=True)
best_clf = clf.fit(X_train,y_train)

In [ ]:
# Training XGBOOST
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
# get best parameters for retraining

pred = best_clf.predict(X_test)
accXGBOOST = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_test)
aucScoreXGBOOST = roc_auc_score(y_test,  y_pred_prob[:,1])
fprXGBOOST, tprXGBOOST, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for XGBOOST is ",aucScoreXGBOOST)
print("Test Accuracy score for XGBOOST is ",accuracy_score(y_test, pred))
predT=best_clf.predict(X_train)
print("Train Accuracy score for XGBOOST is ",accuracy_score(y_train, predT))
print("Best parameters for XGBOOST are ",best_clf.best_params_)

In [71]:
#Gridsearchcv And Training XGBBOST for recall
clfR = GridSearchCV(xgb_model, param_grid = params, scoring = 'recall',cv = skf.split(X_train, y_train),verbose=True)
best_clfR = clfR.fit(X_train,y_train)

Fitting 10 folds for each of 800 candidates, totalling 8000 fits


In [138]:
predR = best_clfR.predict(X_test)
predRT=best_clfR.predict(X_train)
recallXGBOOST=recall_score(y_test, predR)
print("Test Recall score for XGBOOST is ",recallXGBOOST)
print("Train recall score for XGBOOST is ",recall_score(y_train, predRT))
print("Best parameters for recall of XGBOOST are ",best_clfR.best_params_)

Test Recall score for XGBOOST is  0.8704761904761905
Train recall score for XGBOOST is  0.8933764135702746
Best parameters for recall of XGBOOST are  {'colsample_bytree': 0.83, 'eta': 0.20000000000000004, 'gamma': 5, 'min_child_weight': 1.0, 'subsample': 0.94}


XGBOOST gives accuracy of 0.85 which is best amongst them
There is very less difference between train and test accuracy so its not overfitting or underfitting
Recall is 0.88 which is really Good
XGBBOST is basically an implementation of GBDT Gradint Boosting Decision Tree
XGBOOST have given us the best result so far

In [ ]:
#Notes:

XGBOOST gives accuracy of 0.85 which is best amongst them
There is very less difference between train and test accuracy so its not overfitting or underfitting
Recall is 0.88 which is really Good
XGBBOST is basically an implementation of GBDT Gradint Boosting Decision Tree
XGBOOST have given us the best result so far